In [11]:
import os
import json

test = json.load(open('cocofinal_attention_val.json', 'r'))
train = json.load(open('cocofinal_attention_train.json', 'r'))
# print len(test)
# print len(train)

In [12]:
print test[0]
print train[0]

{u'img_id': 519601, u'ques_cap_id': 51960111, u'question': u'Is this train still operating today?', u'caption': u'A train traveling down tracks near a tree filled forest.', u'ques_id': 5196011, u'file_path': u'train2014/COCO_train2014_000000519601.jpg'}
{u'img_id': 314392, u'ques_cap_id': 31439211, u'question': u'What is she drinking?', u'caption': u'A woman standing in a room holding a cup', u'ques_id': 3143921, u'file_path': u'train2014/COCO_train2014_000000314392.jpg'}


In [13]:
import copy
from random import shuffle, seed
import sys
import os.path
import argparse
import glob
import numpy as np
from scipy.misc import imread, imresize
import scipy.io
import pdb
import string
import h5py
from nltk.tokenize import word_tokenize
import json
import nltk

def posfinder(caption):
    wordDict = {}
    tagDict = {}
    tagNSet = set()# noun Set #http://www.computational-logic.org/iccl/master/lectures/summer06/nlp/part-of-speech-tagging.pdf
    tagVSet = set()#verb set
    tagWSet = set()#verb set
    tagPSet = set()#pronoun set
    tagAdSet = set()#Adjective set
    tagAvSet = set()#Adverb set
    tagOSet = set()#other set

    # sentence = "Hello my name is Abhishek Mitra"
    sentence = caption
    sentence = word_tokenize(sentence)
    posTags = nltk.pos_tag(sentence)
#     print('sent',posTags)

#     print [s for s in posTags if s[1] != 'NN' and s[1] != 'NNP']

    for s in posTags:
        if s[1].startswith('NN') or s[1].startswith('NNS') or s[1].startswith('NNP') or s[1].startswith('NNPS'): # for noun
            tagNSet.add(s[0])
        elif s[1].startswith('VB') or s[1].startswith('VBG') or s[1].startswith('VBN') or s[1].startswith('VBP') or s[1].startswith('VBZ') or s[1].startswith('VBD'): #for verb
            tagVSet.add(s[0])
        elif s[1].startswith('WDT') or s[1].startswith('WP') or s[1].startswith('WP$') or s[1].startswith('WRB'): # wh words
            tagWSet.add(s[0])
        elif s[1].startswith('PP') or s[1].startswith('PP$') : # pronoun words
            tagPSet.add(s[0])
        elif s[1].startswith('RB') or s[1].startswith('RBR') or s[1].startswith('RBS') : # adverb words
            tagAvSet.add(s[0])
        elif s[1].startswith('JJ') or s[1].startswith('JJR') or s[1].startswith('JJS'): # Adjective words
            tagAdSet.add(s[0])
        else:    
            tagOSet.add(s[0])
    Nset = list(tagNSet|tagPSet|tagAdSet)
    Vset = list(tagVSet)
    Whset = list(tagWSet)
    count = 0
    noun = ''
    if len(Nset)==0:
        noun = " "
    else:
        while(count<5):
            for j in range(len(Nset)):
                noun = noun + ' ' + Nset[j]
                count += 1
                if count == 5:
                    break
    # print count
#         for j in range(len(Nset)):
#             if count == 5:
#                 break
#             noun = noun + ' ' + Nset[j]
#             count += 1    
    count = 0
    verb = ''
    if len(Vset)==0:
        verb = " "
    else:
        while(count<5):
            for j in range(len(Vset)):
                verb = verb + ' ' + Vset[j]
                count += 1
                if count == 5:
                    break
#     # print count
#         for j in range(len(Vset)):
#             if count == 5:
#                 break
#             verb = verb + ' ' + Vset[j]
#             count += 1
            
    count = 0
    whword = ''
    if len(Whset)==0:
        whword = " "
    else:
        while(count<5):
            for j in range(len(Whset)):
                whword = whword + ' ' + Whset[j]
                count += 1

#             if count == 5:
#                 break
    # print count
#         for j in range(len(Whset)):
#             if count == 5:
#                 break
#             whword = whword + ' ' + Whset[j]
#             count += 1

    
    return noun,verb,whword




In [14]:
print train[0]['caption'][:-1] + ' ' + train[0]['question']
x = posfinder(train[0]['caption'][:-1] + ' ' + train[0]['question'])
print x
print x[0]
wh = 'why how when what who which where '.decode('utf-8')

print wh

A woman standing in a room holding a cu What is she drinking?
(u' woman room cu woman room', u' standing is drinking holding standing', u' What What What What What')
 woman room cu woman room
why how when what who which where 


In [15]:
import nltk
nltk.download('averaged_perceptron_tagger')
img=train
# outermost = {}
out = [] #len(raw_img)
for i in range(0,len(img)):#len(img)
    
    imgid = img[i]['img_id']
    qcapid = img[i]['ques_cap_id']
    quesid = img[i]['ques_id']
    ques = img[i]['question']
    imgpath = img[i]['file_path']
    capt = img[i]['caption']
    noun,verb,whword = posfinder(capt[:-1] + ' ' + ques)
    #     ans = raw_img[i]['ans']
    
    
#     for j in range(0,len(img)):
#         if img[j]['file_name'] == imgpath:
#             capt = img[j]['caption']
    #ques = imgs[i]['questions'].split(u'---')
    #final_imgid = "COCO_img2014_{0:012d}.jpg".format(imgid)
    #print capt
    #loc = 'sab'
    #loc = 'img2014/'

    jimg = {}
    jimg['caption'] = capt
    jimg['question'] = ques
    jimg['img_path'] = imgpath
    jimg['ques_id'] =  quesid
    jimg['ques_cap_id'] =  qcapid
    jimg['image_id'] =  imgid
    jimg['noun'] = noun
    jimg['verb'] = verb
    jimg['whword'] = wh
    #     jimg['ans'] = ans
    #sent= []
    #sent.append(ques)
    out.append(jimg)
#out.append('}')
#print capt
# outermost['questions'] = out
print (len(out))
json.dump(out, open('coco_pos_tv_train.json', 'w'))


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /users/gpu/badri1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
62500


In [16]:
print posfinder('A woman standing next to a pink box with eyes Why you use that purse')
print out[1]

(' pink box eyes woman purse', ' standing use standing use standing', ' Why Why Why Why Why')
{'noun': u' woman door room cup woman', 'img_path': u'train2014/COCO_train2014_000000314392.jpg', 'caption': u'A woman in a room holding a cup and a door behind her', 'ques_cap_id': 31439212, 'question': u'What is she drinking?', 'whword': u'why how when what who which where ', 'image_id': 314392, 'verb': u' is drinking holding is drinking', 'ques_id': 3143921}
